## Imports

In [1]:
import pandas as pd

# import spacy
import numpy as np
import sklearn as skl 
import xgboost as xgb


## Training

In [ ]:
target = 'label'
input_column = 'cleanTitle'

train_data = pd.read_pickle('../../../Files/Submissions/train/sets/train_split_submission.pickle') 
valid_data = pd.read_pickle('../../../Files/Submissions/train/sets/val_split_submission.pickle')
test_data = pd.read_pickle('../../../Files/Submissions/train/sets/test_split_submission.pickle')


In [2]:
target = 'label'
input_column = 'cleanText'

train_data = pd.read_csv('../../../Files/Submissions/train/train2_v2.csv', sep='|') 
# valid_data = pd.read_pi('../../../Files/Submissions/train/sets/val_split_submission.pickle')
test_data = pd.read_csv('../../../Files/Submissions/train/test2_v2.csv', sep='|')


In [3]:
train_data.shape

(1843, 14)

In [4]:
train_data.groupby('label').count()

,id,text,url,score,title,author,selftext,permalink,subreddit,created_utc,num_comments,Comments,cleanText
label,,,,,,,,,,,,,
0,577,577,577,577,577,577,245,577,577,577,577,0,577
1,708,708,708,708,708,708,144,708,708,708,708,1,708
2,558,558,558,558,558,558,237,558,558,558,558,0,558


In [54]:
target = 'label'
input_column = 'cleanText'

train_data = pd.read_csv('../../../Files/Submissions/train/train1.csv', sep='|') 
# valid_data = pd.read_pi('../../../Files/Submissions/train/sets/val_split_submission.pickle')
test_data = pd.read_csv('../../../Files/Submissions/train/test1.csv', sep='|')

In [60]:
test_data.shape

(368, 14)

In [5]:

train_data = train_data[[target, input_column]]
# valid_data = valid_data[[target, input_column]]
test_data = test_data[[target, input_column]]

data = pd.concat([train_data, test_data])


train_instances = train_data[input_column].apply(str).apply(str.split)
train_labels = train_data[target]

# collect known word tokens and tags
wordset, labelset = set(), set()

# collect tags from all data, to prevent unseen labels
labelset.update(set(data[target]))

# get the vocabulary
for words in train_instances:
    wordset.update(set(words))

# map words and tags into ints
PAD = '-PAD-'
UNK = '-UNK-'
word2int = {word: i + 2 for i, word in enumerate(sorted(wordset))}
word2int[PAD] = 0  # special token for padding
word2int[UNK] = 1  # special token for unknown words
 
label2int = {label: i for i, label in enumerate(sorted(labelset))}
# inverted index to translate it back
int2label = {i:label for label, i in label2int.items()}


def convert2ints(instances):
    """
    function to apply the mapping to all words
    """
    result = []
    for words in instances:
        # replace words with int, 1 for unknown words
        word_ints = [word2int.get(word, 1) for word in words]
        result.append(word_ints)
    return result
                          
train_instances_int = convert2ints(train_instances)
train_labels_int = [label2int[label] for label in train_labels]

In [6]:
test_instances = test_data[input_column].apply(str).apply(str.split)
test_labels = test_data[target]

test_instances_int = convert2ints(test_instances)
test_labels_int = [label2int[label] for label in test_labels]

# convert dev data
# val_instances = valid_data[input_column].apply(str).apply(str.split)
# val_labels = valid_data[target]

# val_instances_int = convert2ints(val_instances)
# val_labels_int = [label2int[label] for label in val_labels]

In [7]:
from keras.utils import to_categorical

train_labels_1hot = to_categorical(train_labels_int, len(label2int))
test_labels_1hot = to_categorical(test_labels_int, len(label2int))
# val_labels_1hot = to_categorical(val_labels_int, len(label2int))

train_labels_1hot[0]

array([0., 1., 0.], dtype=float32)

In [8]:
# compute 95th percentile of training sentence lengths
L = sorted(map(len, train_instances))
MAX_LENGTH = L[int(len(L)*0.95)]
print(MAX_LENGTH)

# apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_instances_int = pad_sequences(train_instances_int, padding='post', maxlen=MAX_LENGTH)
test_instances_int = pad_sequences(test_instances_int, padding='post', maxlen=MAX_LENGTH)
# val_instances_int = pad_sequences(val_instances_int, padding='post', maxlen=MAX_LENGTH)

print(train_instances[0], len(train_instances[0]))
print(train_instances_int[0], len(train_instances_int[0]))

107
['covid[NUM', ']', 'un', 'health', 'agency', 'warn', 'new', 'dangerous', 'phase', 'case', 'mount', 'delete'] 12
[1383   29 6629 2830  177 7012 4208 1514 4734  926 4061 1598    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0] 107


In [9]:
print(len(test_instances_int)) 

461


In [10]:
dtrain = xgb.DMatrix(train_instances_int, label=train_labels_int)
# dval = xgb.DMatrix(val_instances_int, label=val_labels_int)
dtest = xgb.DMatrix(test_instances_int, label=test_labels_int)

In [11]:
param = {'max_depth': 8, 'eta': 1, 'objective': 'multi:softmax', 'num_class': 3, 'eval_metric': ['auc', 'ams@0'], 'nthread': 4}

# evallist = [(dval, 'eval'), (dtrain, 'train')]

In [12]:
num_round = 500
seed = 42
cv_results = xgb.cv(
    param, 
    dtrain, 
    num_boost_round=num_round,
    seed=seed, 
    nfold=5,
)


In [13]:
cv_results

,train-auc-mean,train-auc-std,train-ams@0-mean,train-ams@0-std,test-auc-mean,test-auc-std,test-ams@0-mean,test-ams@0-std
0,0.885010,9.678998e-03,37.140437,0.454461,0.615563,0.013389,18.245574,0.782637
1,0.951454,4.908327e-03,37.134097,0.455017,0.640555,0.019723,18.249977,0.883169
2,0.978711,2.133078e-03,37.124389,0.448620,0.649708,0.014573,18.267078,0.861399
3,0.992777,1.137231e-03,37.127290,0.452073,0.654773,0.017680,18.232625,0.848944
4,0.996779,7.659605e-04,37.133949,0.442635,0.655947,0.010941,18.217790,0.848775
...,...,...,...,...,...,...,...,...
495,0.999999,8.542434e-07,37.175172,0.459533,0.688307,0.012679,18.201130,0.873399
496,0.999999,8.542434e-07,37.184064,0.459286,0.688248,0.012657,18.193703,0.876699
497,0.999999,8.542434e-07,37.181373,0.455489,0.688322,0.012652,18.201130,0.873399
498,0.999999,8.542434e-07,37.175986,0.458985,0.688243,0.012609,18.201130,0.873399


## Hyperparameter Tuning

In [14]:
from sklearn.model_selection import GridSearchCV

In [17]:
params = { 'num_class': [2], 
        'max_depth' : [2, 3, 4, 5, 6, 7],
        'learning_rate' : [0.01, 0.05, 0.1],
        'n_estimators' : [100, 500, 1000, 2000],
        'objective': ['multi:softprob'],
        # 'eval_metric': ['roc_auc_ovo'],
        'gamma': [0.1, 0.5, 1, 2, 5],
        'colsample_bytree': [0.5, 0.7, 1]
}

xgbc = xgb.XGBClassifier(seed = 42, )

clf = GridSearchCV(estimator = xgbc,
                param_grid = params,
                scoring='roc_auc_ovo',
                verbose = 1,
                n_jobs = 10
            
)

print('Setup completed, training now')

clf.fit(train_instances_int, train_labels_int)


Setup completed, training now
Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_to_...
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_la

In [18]:
from joblib import dump, load

In [19]:
dump(clf, 'xgbCVGridSearchC1_3.joblib')

['xgbCVGridSearchC1_3.joblib']

In [20]:
model = clf.best_estimator_

dump(model, 'xgbBest_CVC1_3.joblib') 

['xgbBest_CVC1_3.joblib']

## Analysis 

In [21]:
df1 = pd.DataFrame(clf.cv_results_)

In [22]:
df1[df1['mean_test_score'] == df1['mean_test_score'].max()]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_gamma,param_learning_rate,param_max_depth,param_n_estimators,param_num_class,param_objective,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
61,31.92091,0.180993,0.043928,0.004642,0.5,0.1,0.1,5,500,2,multi:softprob,"{'colsample_bytree': 0.5, 'gamma': 0.1, 'learn...",0.726565,0.71223,0.72245,0.736947,0.728121,0.725263,0.008052,1


In [2]:
Gridsearch = load('xgbCVGridSearch.joblib')

In [23]:
model = clf.best_estimator_

In [24]:
type(model)

xgboost.sklearn.XGBClassifier

In [25]:
print("Best parameters:", clf.best_params_)
print("Highest AUC: ", (clf.best_score_))


Best parameters: {'colsample_bytree': 0.5, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 500, 'num_class': 2, 'objective': 'multi:softprob'}
Highest AUC:  0.725262733858856


In [8]:
df1 = pd.DataFrame(Gridesearch.cv_results_)
# df1.to_csv('xgboost_results.csv')

In [10]:
df1.sort_values(['rank_test_score']).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_num_class,param_objective,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
44,80.091854,8.578351,0.422838,0.032511,0.5,0.1,10,1000,3,multi:softprob,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",0.928852,0.929915,0.932881,0.929763,0.933452,0.930973,0.001837,1
41,61.540091,6.199461,0.324311,0.029329,0.5,0.1,8,1000,3,multi:softprob,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",0.927215,0.928086,0.931972,0.927805,0.931833,0.929382,0.002077,2
29,84.471425,9.330232,0.453256,0.045889,0.5,0.05,10,1000,3,multi:softprob,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",0.925553,0.927085,0.930634,0.926786,0.930606,0.928133,0.002095,3
89,92.961855,7.584533,0.471483,0.070459,0.7,0.1,10,1000,3,multi:softprob,"{'colsample_bytree': 0.7, 'learning_rate': 0.1...",0.926387,0.926844,0.930113,0.926399,0.930192,0.927987,0.001776,4
43,45.372425,1.339221,0.232754,0.016557,0.5,0.1,10,500,3,multi:softprob,"{'colsample_bytree': 0.5, 'learning_rate': 0.1...",0.925062,0.926473,0.929667,0.926238,0.930366,0.927561,0.002073,5


In [11]:
df1.to_csv('xgboost_results.csv')

In [26]:
proba = model.predict_proba(test_instances_int)

In [14]:
test = model.predict(test_instances_int)

In [27]:
proba.tofile('../../../Files/models/xgb_proba_v3.txt', sep=' ')

In [21]:
test.tofile('../../../Files/models/xgb_test.txt', sep=' ')

## Test trained model

In [48]:
model = load('xgbBest_CVC2.joblib')

In [50]:
y_test = model.predict_proba(test_instances_int)

In [51]:
y_test.tofile('../../../Files/models/xgboostC2.txt', sep=' ')

In [35]:
y_test.shape

(2743, 2)